# MuJoCo入门篇04. 代码设计的方法论Methodology
https://zhuanlan.zhihu.com/p/705594503

In [1]:
"""
这个例子的参考是https://github.com/tayalmanan28/MuJoCo-Tutorial/blob/main/tutorial/tutorial_6.ipynb
,和官方文档是等效的, 具体章节是tutorial6-Tendons, actuators and sensors
"""
import mujoco
import mediapy as media
import matplotlib.pyplot as plt
import time
import os
import itertools
import numpy as np

rel_path = "eg01p5_5_scene_with_manipulator.xml"
dir_name = os.path.dirname(__file__) if '__file__' in globals() else os.getcwd()
xml_path = os.path.join(dir_name + '\\' + rel_path)
print(xml_path)
model = mujoco.MjModel.from_xml_path(xml_path)
data = mujoco.MjData(model)

n_frames = 60
height, width, fps = 480, 640, 60
frames = []
times = []
sensordata = []

renderer = mujoco.Renderer(model, height, width)
mujoco.mj_resetData(model, data)

# simulate and render 1 for gripper closing
data.ctrl[0] = 80 #constant actuator signal, tendon length, +: open gripper, -: close gripper
data.ctrl[1] = 0.1 #constant actuator signal
data.ctrl[[2, 3, 4]] = 0, 0, 0

for i in range(2*n_frames):
  while data.time < i/fps:#time的micro step应该比1/fps更精细,所以在一帧之内,需要step多次, make sense. 
    mujoco.mj_step(model, data)
    times.append(data.time)
    # sensordata.append(data.sensor('accelerometer').data.copy())
  renderer.update_scene(data, "fixed_2")
  frame = renderer.render()
  frames.append(frame)
  
# simulate and render 2 for gripper opening
data.ctrl[0] = 80 #keep open
data.ctrl[1] = -0.28 #moving down
data.ctrl[[2, 3, 4]] = 0, 0, 0

for j in range(2*n_frames, 4*n_frames):
  while data.time < j/fps:
    mujoco.mj_step(model, data)
    times.append(data.time)
  renderer.update_scene(data, "fixed_2")
  frame = renderer.render()
  frames.append(frame)

# simulate and render 3
data.ctrl[0] = -60 #close gripper
data.ctrl[1] = -0.28 #hold position
data.ctrl[[2, 3, 4]] = 0, 0, 0


for k in range(4*n_frames, 6*n_frames):
  while data.time < k/fps:
    mujoco.mj_step(model, data)
    times.append(data.time)
  renderer.update_scene(data, "fixed_2")
  frame = renderer.render()
  frames.append(frame)

# simulate and render 4
data.ctrl[0] = -60 #keep closed
data.ctrl[1] = 0.5 #move up
data.ctrl[[2, 3, 4]] = 0, 0, 0

for l in range(6*n_frames, 8*n_frames):
  while data.time < l/fps:
    mujoco.mj_step(model, data)
    times.append(data.time)
  renderer.update_scene(data, "fixed_2")
  frame = renderer.render()
  frames.append(frame)

# simulate and render 5
data.ctrl[0] = -60 #keep closed
data.ctrl[1] = 0.5 #hold pos
data.ctrl[[2, 3, 4]] = 0, 1.4, 0

for m in range(8*n_frames, 10*n_frames):
  while data.time < m/fps:
    mujoco.mj_step(model, data)
    times.append(data.time)
  renderer.update_scene(data, "fixed_2")
  frame = renderer.render()
  frames.append(frame)

# simulate and render 6
data.ctrl[0] = -60 #keep closed
data.ctrl[1] = 0.5 #hold pos
data.ctrl[[2, 3, 4]] = 0, -1.0, 0

for m in range(10*n_frames, 12*n_frames):
  while data.time < m/fps:
    mujoco.mj_step(model, data)
    times.append(data.time)
  renderer.update_scene(data, "fixed_2")
  frame = renderer.render()
  frames.append(frame)

# simulate and render 7
data.ctrl[0] = -60 #keep closed
data.ctrl[1] = 0.5 #hold pos
data.ctrl[[2, 3, 4]] = 0, 0, 0

for m in range(12*n_frames, 14*n_frames):
  while data.time < m/fps:
    mujoco.mj_step(model, data)
    times.append(data.time)
  renderer.update_scene(data, "fixed_2")
  frame = renderer.render()
  frames.append(frame)

# simulate and render 8
data.ctrl[0] = 0 #release gripper
data.ctrl[1] = 0.5 #hold pos
data.ctrl[[2, 3, 4]] = 0, 0, 0

for m in range(14*n_frames, 18*n_frames):
  while data.time < m/fps:
    mujoco.mj_step(model, data)
    times.append(data.time)
  renderer.update_scene(data, "fixed_2")
  frame = renderer.render()
  frames.append(frame)
    
media.show_video(frames, fps=fps)


c:\Users\Wenbin\Documents\GitHub\MuJoCo-Tutorial-fork\tutorial\eg01_p5_5_sw2urdf\urdf\eg01p5_5_scene_with_manipulator.xml


This browser does not support the video tag.


In [2]:
# putting the above sim to mujoco viwer instead of mujoco renderer
import mujoco
import mujoco.viewer
import time

MJCF = """
<mujoco>
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
     rgb2=".2 .3 .4" width="300" height="300" mark="none"/>
    <material name="grid" texture="grid" texrepeat="6 6"
     texuniform="true" reflectance=".2"/>
     <material name="wall" rgba='.5 .5 .5 1'/>
  </asset>

  <default>
    <geom type="box" size=".05 .05 .05" />
    <joint type="free"/>
  </default>

  <worldbody>
    <light name="light" pos="-.2 0 1"/>
    <geom name="ground" type="plane" size=".5 .5 10" material="grid"
     zaxis="-.3 0 1" friction=".1"/> 
    <camera name="y" pos="-.1 -.6 .3" xyaxes="1 0 0 0 1 2"/>
    <body pos="0 0 .1">
      <joint/>
      <geom/>
    </body>
    <body pos="0 .2 .1">
      <joint/>
      <geom friction=".33"/>
    </body>
  </worldbody>

</mujoco>
"""

# Load model
model = mujoco.MjModel.from_xml_string(MJCF)
data = mujoco.MjData(model)

# Initialize the viewer
with mujoco.viewer.launch_passive(model, data) as viewer:
    # Reset simulation
    mujoco.mj_resetData(model, data)    
    # Simulation loop
    start = time.time()
    while viewer.is_running():
        step_start = time.time()        
        # Step simulation
        mujoco.mj_step(model, data)        
        # Update viewer
        viewer.sync()        
        # Control simulation rate to real time
        time_until_next_step = model.opt.timestep - (time.time() - step_start)
        if time_until_next_step > 0:
            time.sleep(time_until_next_step)

In [1]:
import mujoco as mj
from mujoco.glfw import glfw
import mujoco.viewer
import numpy as np
import time
import os


class GraspControl:
    grasp_ongoing = False
    def __init__(self, filename, is_show): #对应伪代码 initVisualData();
        # 1. model and data 加载模型与数据
        self.model = mj.MjModel.from_xml_path(filename)
        self.data = mj.MjData(self.model)
        self.is_show = is_show
        if self.is_show:
            self.viewer = mujoco.viewer.launch_passive(self.model, self.data, key_callback=self.keyboard_cb, show_left_ui = True, show_right_ui = False)
            self.viewer.opt.frame = mj.mjtFrame.mjFRAME_CAMERA
            # self.viewer.cam.type = 'mjCAMERA_FIXED'
            # self.viewer.cam.fixedcamid = 0


    def init_controller(self): #对应伪代码 initControlData();
        # 1. set init pos, vel 设置初始化条件(位置和速度)
        # self.data.qpos[0] = 0.0
            # self.data.qvel[0] = 0.0
            # self.data.qvel[1] = 0.0
            # self.data.qvel[2] = 0.0
            # self.data.qvel[3] = 0.0
            # self.data.qvel[4] = 0.0
            # self.data.qvel[5] = 0.0
        # 2. set the controller, 加载控制器
        mj.set_mjcb_control(self.controller_UpOpen)

    def controller_UpOpen(self, model, data): #对应伪代码 callback function (此处按照controller与visualization所规定的callback函数进行补充)
        data.ctrl[0] = 80 #constant actuator signal, tendon length, +: open gripper, -: close gripper
        data.ctrl[1] = 0.1 #constant actuator signal
        data.ctrl[[2, 3, 4]] = 0, 0, 0

    def controller_Grasp(self, model, data):
        if not grasp_ongoing: 
            start_time = time.time()
        grasp_ongoing = True

        if (time.time() - start_time) < 2:
            data.ctrl[0] = 80 #constant actuator signal, tendon length, +: open gripper, -: close gripper
            data.ctrl[1] = -0.27 #constant actuator signal
            data.ctrl[[2, 3, 4]] = 0, 0, 0
            print("time<2")

        if (time.time() - start_time) > 2:
            data.ctrl[0] = -60 #constant actuator signal, tendon length, +: open gripper, -: close gripper
            # data.ctrl[1] = -0.27 #constant actuator signal
            data.ctrl[[2, 3, 4]] = 0, 0, 0
            print("time>2")
        if (time.time() - start_time) > 4:
            data.ctrl[0] = -60 #constant actuator signal, tendon length, +: open gripper, -: close gripper
            data.ctrl[1] = 0.6 #constant actuator signal
            data.ctrl[[2, 3, 4]] = 0, 0, 0
            print("time>4")

    def controller_Release(self, model, data):
        data.ctrl[0] = 0 #constant actuator signal, tendon length, +: open gripper, -: close gripper
        # data.ctrl[1] = -0.27 #constant actuator signal
        data.ctrl[[2, 3, 4]] = 0, 0, 0
    def controller_Mani_swing_Y(self, model, data):
        data.ctrl[[2, 3, 4]] = 0, 1.0, 0
    def controller_Mani_swing_Y_(self, model, data):
        data.ctrl[[2, 3, 4]] = 0, -1.0, 0

    def main(self): 
        """
        对应伪代码 realtime simulation,  
        while(...) {
        updateControlData();
        updateVisualData(); }
        """
        sim_start, sim_end = time.time(), 50.0
        while time.time() - sim_start < sim_end:
            step_start = time.time()
            loop_num, loop_count = 50, 0
            # 1. running for 0.002*50 = 0.1s
            while loop_count < loop_num:
                loop_count = loop_count + 1
                mj.mj_step(self.model, self.data)
            # 2. GUI show
            if self.is_show:
                if self.viewer.is_running():
                    self.viewer.sync()
                else:
                    break
            # 3. sleep for next period
            step_next_delta = self.model.opt.timestep * loop_count - (time.time() - step_start)
            if step_next_delta > 0:
                time.sleep(step_next_delta)
        if self.is_show: 
            """
            对应伪代码 4. end simulation, 
            deleteControlData(); 
            deleteVisualData();
            """
            self.viewer.close()

    def keyboard_cb(self, keycode):
        # Print keycode for debugging
        print(f"Received keycode: {keycode}")
        
        # Handle space bar
        if keycode == 32:  # Space bar
            mj.mj_forward(self.model, self.data)
            self.init_controller()
        
        # Arrow keys using direct ASCII values
        elif keycode == 265:  # Up arrow
            print("Up pressed")
            mj.mj_forward(self.model, self.data)
            mj.set_mjcb_control(self.controller_Grasp)
            
        elif keycode == 264:  # Down arrow
            print("Down pressed")
            mj.mj_forward(self.model, self.data)
            mj.set_mjcb_control(self.controller_Release)
            
        elif keycode == 263:  # Left arrow
            print("Left pressed")
            mj.mj_forward(self.model, self.data)
            mj.set_mjcb_control(self.controller_Mani_swing_Y)
           
        elif keycode == 262:  # Right arrow
            print("Right pressed")
            mj.mj_forward(self.model, self.data)
            mj.set_mjcb_control(self.controller_Mani_swing_Y_)

if __name__ == "__main__":
    rel_path = "eg01p5_5_scene_with_manipulator.xml"
    # dir_name = os.path.dirname(__file__)
    dir_name = os.path.dirname(__file__) if '__file__' in globals() else os.getcwd()
    xml_path = os.path.join(dir_name + "\\" + rel_path)
    print(xml_path)
    is_show = True
    GraspControl = GraspControl(xml_path, is_show)
    GraspControl.main()



c:\Users\Wenbin\Documents\GitHub\MuJoCo-Tutorial-fork\tutorial\eg01_p5_5_sw2urdf\urdf\eg01p5_5_scene_with_manipulator.xml
Received keycode: 265
Up pressed


UnboundLocalError: cannot access local variable 'grasp_ongoing' where it is not associated with a value

In [2]:
def keyboard_cb(self, keycode):
    # Print keycode for debugging
    print(f"Received keycode: {keycode}")
    
    # Handle space bar
    if keycode == 32:  # Space bar
        mj.mj_forward(self.model, self.data)
        self.init_controller()
    
    # Arrow keys using direct ASCII values
    elif keycode == 265:  # Up arrow
        print("Up pressed")
        mj.mj_forward(self.model, self.data)
        # Add movement logic here
        
    elif keycode == 264:  # Down arrow
        print("Down pressed")
        mj.mj_forward(self.model, self.data)
        # Add movement logic here
        
    elif keycode == 263:  # Left arrow
        print("Left pressed")
        mj.mj_forward(self.model, self.data)
        # Add movement logic here
        
    elif keycode == 262:  # Right arrow
        print("Right pressed")
        mj.mj_forward(self.model, self.data)
        # Add movement logic here
        
    # Alternative: WASD controls
    elif keycode == ord('w'):  # W key
        print("W pressed")
        mj.mj_forward(self.model, self.data)
        
    elif keycode == ord('s'):  # S key
        print("S pressed")
        mj.mj_forward(self.model, self.data)
        
    elif keycode == ord('a'):  # A key
        print("A pressed")
        mj.mj_forward(self.model, self.data)
        
    elif keycode == ord('d'):  # D key
        print("D pressed")
        mj.mj_forward(self.model, self.data)

In [2]:
import mujoco
import numpy as np
import time

# Define model with a rotational joint
xml = """
<mujoco>
    <worldbody>
        <body name="base" pos="0 0 0">
            <joint name="rotate_joint" type="hinge"/>
            <geom type="cylinder" size="0.1 0.02" rgba="1 0 0 1"/>
            <geom type="box" pos="0.1 0 0" size="0.1 0.02 0.02" rgba="0 1 0 1"/>
        </body>
    </worldbody>
    <actuator>
        <position joint="rotate_joint" name="pos_actuator" kp="1"/>
    </actuator>
</mujoco>
"""

# Load model and create data
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

def degrees_to_radians(degrees):
    return degrees * np.pi / 180.0

# Define movement sequence
class MovementSequence:
    def __init__(self):
        self.sequence = [
            {"target": 180, "duration": 2.0},  # Rotate 180 degrees
            {"target": 135, "duration": 1.0},  # Go back 45 degrees
            {"target": 180, "duration": 1.0},  # Go forward 45 degrees
            {"target": 225, "duration": 1.0},  # Go forward another 45 degrees
        ]
        self.current_step = 0
        self.step_time = 0
        
    def get_target(self, current_time):
        if self.current_step >= len(self.sequence):
            return self.sequence[-1]["target"]
            
        step = self.sequence[self.current_step]
        
        # Check if it's time to move to next step
        if self.step_time >= step["duration"]:
            self.current_step += 1
            self.step_time = 0
            if self.current_step < len(self.sequence):
                return self.sequence[self.current_step]["target"]
            return self.sequence[-1]["target"]
            
        self.step_time += current_time
        return step["target"]

# Create movement sequence
movement = MovementSequence()
sim_time = 0.0
dt = 0.01  # Simulation timestep

# Simulation loop
while sim_time < 10.0:  # Run for 10 seconds
    # Get target angle for current time
    target_degrees = movement.get_target(dt)
    target_radians = degrees_to_radians(target_degrees)
    
    # Set control signal
    data.ctrl[0] = target_radians
    
    # Step simulation
    mujoco.mj_step(model, data)
    
    # Print current state
    current_degrees = data.qpos[0] * 180 / np.pi
    print(f"Time: {sim_time:.2f}s, Target: {target_degrees:.1f}°, Current: {current_degrees:.1f}°")
    
    sim_time += dt
    time.sleep(dt)

Time: 0.00s, Target: 180.0°, Current: 0.1°
Time: 0.01s, Target: 180.0°, Current: 0.2°
Time: 0.02s, Target: 180.0°, Current: 0.4°
Time: 0.03s, Target: 180.0°, Current: 0.7°
Time: 0.04s, Target: 180.0°, Current: 1.0°
Time: 0.05s, Target: 180.0°, Current: 1.4°
Time: 0.06s, Target: 180.0°, Current: 1.9°
Time: 0.07s, Target: 180.0°, Current: 2.4°
Time: 0.08s, Target: 180.0°, Current: 3.1°
Time: 0.09s, Target: 180.0°, Current: 3.7°
Time: 0.10s, Target: 180.0°, Current: 4.5°
Time: 0.11s, Target: 180.0°, Current: 5.3°
Time: 0.12s, Target: 180.0°, Current: 6.2°
Time: 0.13s, Target: 180.0°, Current: 7.1°
Time: 0.14s, Target: 180.0°, Current: 8.1°
Time: 0.15s, Target: 180.0°, Current: 9.2°
Time: 0.16s, Target: 180.0°, Current: 10.3°
Time: 0.17s, Target: 180.0°, Current: 11.5°
Time: 0.18s, Target: 180.0°, Current: 12.8°
Time: 0.19s, Target: 180.0°, Current: 14.1°
Time: 0.20s, Target: 180.0°, Current: 15.5°
Time: 0.21s, Target: 180.0°, Current: 16.9°
Time: 0.22s, Target: 180.0°, Current: 18.4°
Time

In [1]:
import mujoco
import mujoco.viewer
import numpy as np
import time

# # Define model with a rotational joint
# xml = """
# <mujoco>
#     <worldbody>
#         <body name="link">
#             <joint name="rotate_joint" type="hinge"/>
#             <geom type="cylinder" size="0.1 0.1" rgba="1 0 0 1"/>
#             <!-- Visual indicator for rotation -->
#             <geom type="capsule" size="0.02 0.2" pos="0.2 0 0" rgba="0 1 0 1"/>
#         </body>
#     </worldbody>

#     <actuator>
#         <position 
#             joint="rotate_joint" 
#             kp="10"
#             ctrlrange="-6.28319 6.28319"  <!-- ±2π -->
#             forcerange="-100 100"
#         />
#     </actuator>
# </mujoco>
# """
# Define model with a rotational joint
xml = """
<mujoco>
    <worldbody>
        <body name="base" pos="0 0 0">
            <joint name="rotate_joint" type="hinge"/>
            <geom type="cylinder" size="0.1 0.02" rgba="1 0 0 1"/>
            <geom type="box" pos="0.1 0 0" size="0.1 0.02 0.02" rgba="0 1 0 1"/>
        </body>
    </worldbody>
    <actuator>
        <position joint="rotate_joint" name="pos_actuator" kp="1"/>
    </actuator>
</mujoco>
"""

def deg_to_rad(deg):
    return deg * np.pi / 180.0

def normalize_angle_difference(target, current):
    diff = target - current
    return (diff + np.pi) % (2 * np.pi) - np.pi

def move_to_position(model, data, target_deg, threshold=0.01):
    target_rad = deg_to_rad(target_deg)
    
    # Create viewer
    with mujoco.viewer.launch_passive(model, data) as viewer:
        while viewer.is_running():
            # Get current position
            current_pos = data.qpos[0]
            
            # Calculate normalized difference
            diff = normalize_angle_difference(target_rad, current_pos)
            
            # Check if reached target
            if abs(diff) < threshold:
                print(f"Reached {target_deg} degrees")
                time.sleep(1)  # Wait a second at target
                break
                
            # Apply control
            data.ctrl[0] = current_pos + diff
            
            # Step simulation
            mujoco.mj_step(model, data)
            viewer.sync()
            time.sleep(0.01)  # Slow down for visualization

def main():
    # Load your MJCF model
    model = mujoco.MjModel.from_xml_string(xml)
    data = mujoco.MjData(model)
    
    # Movement sequence
    print("Moving to 180 degrees...")
    move_to_position(model, data, 180)
    
    print("Moving back 45 degrees (to 135 degrees)...")
    move_to_position(model, data, 135)

if __name__ == "__main__":
    main()

Moving to 180 degrees...
Moving back 45 degrees (to 135 degrees)...
Reached 135 degrees
